In [1]:
import os

# Ensure the output directory exists
output_dir = "articles"
os.makedirs(output_dir, exist_ok=True)

# Sample content for the articles
article_templates = [
    "Elon Musk is the CEO of {company}. {company} is known for {description}.",
    "{author} is a renowned author known for the book {book_title}. The book explores {theme}.",
    "The {event} of {year} was a significant moment in history, marking {impact}.",
    "{technology} has revolutionized the field of {field}. It has applications in {applications}.",
    "The {animal} is native to {region} and is known for its {traits}.",
]

companies = ["Tesla", "SpaceX", "Neuralink", "The Boring Company"]
descriptions = ["electric vehicles", "space exploration", "neural technology", "tunnel construction"]
authors = ["George Orwell", "Isaac Asimov", "J.K. Rowling", "Agatha Christie"]
books = ["1984", "Foundation", "Harry Potter", "Murder on the Orient Express"]
themes = ["totalitarianism", "future of humanity", "magic", "mystery"]
events = ["Moon Landing", "Fall of the Berlin Wall", "Y2K", "COVID-19 pandemic"]
years = ["1969", "1989", "2000", "2020"]
impacts = ["technological triumph", "end of Cold War", "global awareness", "global health crisis"]
technologies = ["Artificial Intelligence", "Blockchain", "Quantum Computing", "Genetic Engineering"]
fields = ["medicine", "finance", "computing", "agriculture"]
applications = ["diagnostics", "cryptocurrency", "simulations", "crop modification"]
animals = ["Panda", "Kangaroo", "Penguin", "Elephant"]
regions = ["China", "Australia", "Antarctica", "Africa"]
traits = ["gentleness", "jumping ability", "endurance", "memory"]

# Generate 100 files with varying content
for i in range(1, 101):
    article_content = article_templates[i % len(article_templates)].format(
        company=companies[i % len(companies)],
        description=descriptions[i % len(descriptions)],
        author=authors[i % len(authors)],
        book_title=books[i % len(books)],
        theme=themes[i % len(themes)],
        event=events[i % len(events)],
        year=years[i % len(years)],
        impact=impacts[i % len(impacts)],
        technology=technologies[i % len(technologies)],
        field=fields[i % len(fields)],
        applications=applications[i % len(applications)],
        animal=animals[i % len(animals)],
        region=regions[i % len(regions)],
        traits=traits[i % len(traits)],
    )
    file_name = os.path.join(output_dir, f"article_{i:03}.txt")
    with open(file_name, "w") as file:
        file.write(article_content)

print("100 article files generated successfully.")


100 article files generated successfully.


In [2]:
pip install transformers datasets torch faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer
from datasets import Dataset
import torch
import faiss
import os
import numpy as np

# Load the GPT-2 model and tokenizer
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load the DPR question and context encoders
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Load the generated article files
article_dir = "articles"
documents = []
for filename in sorted(os.listdir(article_dir)):
    with open(os.path.join(article_dir, filename), 'r') as file:
        documents.append(file.read())

# Convert the documents into a dataset
dataset = Dataset.from_dict({"text": documents})

# Encode the documents using the context encoder
context_embeddings = []
for doc in documents:
    inputs = context_tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embedding = context_encoder(**inputs).pooler_output.numpy()
    context_embeddings.append(embedding)

# Convert to numpy array and build the FAISS index
context_embeddings = np.vstack(context_embeddings)
index = faiss.IndexFlatIP(context_embeddings.shape[1])
index.add(context_embeddings)


model.safetensors:  80%|########  | 440M/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
def retrieve_documents(question, top_k=1):
    """
    Retrieve top-k documents relevant to the question using FAISS.
    """
    # Encode the question
    inputs = question_tokenizer(question, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        question_embedding = question_encoder(**inputs).pooler_output.numpy()

    # Perform the retrieval
    distances, indices = index.search(question_embedding, top_k)

    # Return the top-k documents
    return [documents[idx] for idx in indices[0]]

def generate_answer(question, retrieved_docs, max_length=150):
    """
    Generate an answer using GPT-2 based on the retrieved documents.
    """
    # Combine the retrieved documents with the question to form the prompt
    prompt = "Context: " + " ".join(retrieved_docs) + f" Question: {question}\nAnswer:"

    # Encode the input and generate the response
    inputs = gpt2_tokenizer.encode(prompt, return_tensors="pt")
    outputs = gpt2_model.generate(inputs, max_length=max_length, num_return_sequences=1)

    # Decode the response
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()

def rag_system(question, top_k=3):
    """
    RAG system that retrieves documents and then generates an answer.
    """
    retrieved_docs = retrieve_documents(question, top_k=top_k)
    response = generate_answer(question, retrieved_docs)
    return response


In [6]:
questions = [
    "What company is Elon Musk the CEO of?",
    "Which company is known for manufacturing electric vehicles?",
    "Who wrote the book '1984'?",
    "In which year did the Moon Landing occur?",
]

# Test the RAG system
for question in questions:
    answer = rag_system(question)
    print(f"Q: {question}\nA: {answer}\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What company is Elon Musk the CEO of?
A: SpaceX.
Question



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Which company is known for manufacturing electric vehicles?
A: Tesla.
Question



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Who wrote the book '1984'?
A: George Orwell

Q: In which year did the Moon Landing occur?
A: The Moon Landing of 1969 was a significant moment in history, marking technological triumph. The Moon Landing of 1969 was a significant



In [ ]:
#with 1000 files

In [2]:
!pip install datasets transformers faiss-gpu torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [5]:
import torch
from datasets import load_dataset
import faiss
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer
import os

# Ensure CUDA is available and set the device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the save directory to your Google Drive location
save_directory = '/content/drive/MyDrive/testPredictions/'
os.makedirs(save_directory, exist_ok=True)

# Load the Wikipedia dataset using streaming and trusting the remote code
dataset = load_dataset('wikipedia', '20220301.en', split='train', streaming=True, trust_remote_code=True)

# Filter to get only a subset (e.g., 1000 articles)
subset = dataset.take(1000)
documents = [doc['text'] for doc in subset]

# Load DPR models and tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Load GPT-2 model and tokenizer
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set maximum length for tokenization
MAX_LENGTH = 512

# Encode the documents using the context encoder and store them in FAISS
context_embeddings = []
for doc in documents:
    # Tokenize and move inputs to GPU
    inputs = context_tokenizer(doc, return_tensors="pt", truncation=True, padding="max_length", max_length=MAX_LENGTH).to(device)

    # Encode using the context encoder and move the output back to CPU to store in FAISS
    with torch.no_grad():
        embedding = context_encoder(**inputs).pooler_output.cpu().numpy()

    context_embeddings.append(embedding)

# Convert to numpy array and build the FAISS index
context_embeddings = np.vstack(context_embeddings)
index = faiss.IndexFlatIP(context_embeddings.shape[1])

# Check if FAISS GPU is available
if hasattr(faiss, 'StandardGpuResources'):
    res = faiss.StandardGpuResources()  # Use a single GPU
    index = faiss.index_cpu_to_gpu(res, 0, index)

index.add(context_embeddings)

# Transfer the index back to the CPU before saving
index_cpu = faiss.index_gpu_to_cpu(index)

# Save the DPR Question Encoder and Tokenizer
question_encoder_path = os.path.join(save_directory, "dpr_question_encoder")
question_encoder.save_pretrained(question_encoder_path)
question_tokenizer.save_pretrained(question_encoder_path)

# Save the DPR Context Encoder and Tokenizer
context_encoder_path = os.path.join(save_directory, "dpr_context_encoder")
context_encoder.save_pretrained(context_encoder_path)
context_tokenizer.save_pretrained(context_encoder_path)

# Save the GPT-2 model and tokenizer
gpt2_model_path = os.path.join(save_directory, "gpt2_model")
gpt2_model.save_pretrained(gpt2_model_path)
gpt2_tokenizer.save_pretrained(gpt2_model_path)

# Save the FAISS index (now on CPU)
faiss_index_path = os.path.join(save_directory, "faiss_index.index")
faiss.write_index(index_cpu, faiss_index_path)

print(f"Models and FAISS index saved to {save_directory}")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

Models and FAISS index saved to /content/drive/MyDrive/testPredictions/


In [13]:
import torch

# Set the pad token to the EOS token for GPT-2
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Function to retrieve documents with their indices
def retrieve_documents(question, top_k=1):
    """
    Retrieve top-k documents relevant to the question using FAISS.
    """
    # Tokenize the question and move inputs to GPU
    inputs = question_tokenizer(question, return_tensors="pt", truncation=True, padding=True).to(device)

    # Encode the question using the DPR question encoder
    with torch.no_grad():
        question_embedding = question_encoder(**inputs).pooler_output.cpu().numpy()

    # Perform a search on the FAISS index
    distances, indices = index_cpu.search(question_embedding, top_k)

    # Retrieve the top-k documents and their indices based on the search results
    retrieved_docs = [documents[idx] for idx in indices[0]]
    retrieved_indices = [idx for idx in indices[0]]

    return retrieved_docs, retrieved_indices

# Function to generate an answer with adjusted parameters
def generate_answer(question, retrieved_docs, max_length=150):
    """
    Generate an answer using GPT-2 based on the retrieved documents.
    """
    # Create the prompt by combining the context from retrieved documents with the question
    prompt = "Context: " + " ".join(retrieved_docs)[:1024] + f" Question: {question}\nAnswer:"  # Truncate to 1024 tokens

    # Tokenize the prompt and move it to GPU
    inputs = gpt2_tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=1024).to(device)

    # Generate an answer using GPT-2 with adjusted parameters
    with torch.no_grad():
        outputs = gpt2_model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_length,
            num_return_sequences=1,
            temperature=0.7,  # Adjusting temperature for less repetitive text
            top_p=0.9,        # Nucleus sampling
            top_k=50,         # Top-k sampling
            repetition_penalty=1.2  # Adding a repetition penalty
        )

    # Decode the generated answer
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return the answer, removing the prompt part
    return answer.split("Answer:")[-1].strip()

# Function to run the RAG system
def rag_system(question, top_k=3):
    """
    RAG system that retrieves documents and then generates an answer.
    """
    # Retrieve the most relevant documents and their indices
    retrieved_docs, retrieved_indices = retrieve_documents(question, top_k=top_k)

    # Generate an answer based on the retrieved documents
    response = generate_answer(question, retrieved_docs)

    return response, retrieved_indices

# Example usage
questions = [
    "What is the capital city of the country discussed in one of the articles?",
    "Which historical event is covered in the articles, and what year did it occur?",
    "What scientific discovery or invention is explained in one of the articles?",
    "Who is the scientist or inventor mentioned, and what are they known for?",
    "Which cultural tradition or practice is described in the articles?",
    "What is the significance of the festival or holiday mentioned in the articles?",
    "Who is the notable person featured in the articles, and what are their major achievements?",
    "Which artist or author is discussed, and what is their most famous work?",
    "Which government system or political ideology is covered in the articles?",
    "Who is the political leader mentioned in the articles, and what is their role?"
]

# Iterate over each question, retrieve the answer using the RAG system, and print the results
for question in questions:
    answer, article_indices = rag_system(question)
    print(f"Q: {question}\nA: {answer}")
    print(f"Related article numbers: {article_indices}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What is the capital city of the country discussed in one of the articles?
A: The Capital City was founded on September 1535 as part forts into which it had been built during its reign from 1710 until 1803 when they were taken over after being occupied since then.[1] The name "Capital" refers specifically not only towards this period but also toward that time itself; however, there are many other names such like 'City' or 'State'. In fact, some people refer both terms interchangeably,[2][3]. It would be interesting if we could find out what these different meanings mean within each article so far![4]: [5] This question has already appeared before here:[6], where you can see how much information about our topic will help us understand your questions better than others do...
Related article numbers: [937, 641, 216]



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Which historical event is covered in the articles, and what year did it occur?
A: The question was asked during a debate between two members from different political parties about whether or not they should be allowed to speak freely after their death.[1] Both candidates were given an opportunity to answer this one before being removed for good measure; however, both men had been elected without any formal vote cast so that no official decision could take place until later years when there would be more time left over if necessary. In addition some people have suggested using "the" word instead because many historians believe such words are used only once throughout history which may indicate something else than truthfulness rather than just meaning 'truth'. However, these days we do know how much information can change our minds with each passing day - especially since most modern scholars agree upon certain facts based
Related article numbers: [666, 989, 644]



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Q: Who is the notable person featured in the articles, and what are their major achievements?
A: Alfred de la Rochefoucauld wrote The Great War from 1789 until 1914 when it became known that Napoleon had been defeated during World Wars II. This article will show how this war story came about because there were no other sources available which could have provided such an accurate account or even provide any information regarding its origins before 1917! It should not come off like some sort 'historical' piece but rather one based on historical facts instead!!

 (Source: Wikipedia ) A few months ago we published our first collection entitled What Is History?, featuring interviews with various historians who worked under him including Jean Jacques Rousseau - author/editor of La République et lui dans le répondent des sciences du Paris :
Related article numbers: [239, 869, 236]



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Which artist or author is discussed, and what is their most famous work?
A: A master's degree from one of England College University Press' prestigious universities; Bachelor degrees at Cambridge Business School with emphasis on social sciences/economics & psychology / law ; Phd Degree awarded annually through its Institute of Social Sciences. In addition, Masters are required before they can be considered professional artists - those that have achieved some level beyond basic artistic skills may not qualify under these criteria if there was no prior experience working professionally within any profession outside academia : Professional Artists include all types including students studying abroad where it would normally take years after graduation time between studies so long-term education does exist without having been taught anything else about music theory etc., although many people do still consider themselves professionals when applying towards careers elsewhere...Professional

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Which government system or political ideology is covered in the articles?
A: The main focus of this article will primarily deal specifically about how anarchists view their own society today; however there may still exist some misconceptions regarding what they consider "authoritarian" social structures such an authoritarian one would have been if not based on traditional ideas like monarchy/statehood etc. The following sections discuss various aspects related towards these issues including those relating directly into politics but being more general than just monarchist systems where people can choose between different types depending upon who's ruling them - eg military dictatorships vs democratic governments / anarchoarchists versus non-"democratic" ones ie democrats & fascists. In addition we'll look at other areas within our understanding so you don't get confused by any particular terminology used here!

 Question: What does '
Related article numbers: [0, 226, 298]

Q: Who is 